In [ ]:
# 1. Gerekli Kütüphanelerin Yüklenmesi
!pip install -q -U transformers datasets accelerate peft bitsandbytes trl huggingface_hub
!pip install git+https://github.com/huggingface/transformers.git

# 2. Hugging Face Girişi
from huggingface_hub import notebook_login
notebook_login()  # Run this and provide your token

# 3. Model ve Tokenizer Yükleme
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Gemma3TextConfig, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import os

model_name = "google/gemma-3-1b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)  # Token is handled by notebook_login
config = Gemma3TextConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation="eager",  # Recommended for Gemma3
)

# 4. Veri Setini Yükleme ve Formatlama
from datasets import load_dataset
dataset = load_dataset("zinderud/risale")

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# 5. Eğitim Konfigürasyonu
output_dir = "outputs"
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    max_steps=500,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=10,
    output_dir=output_dir,
    optim="paged_adamw_8bit",
    report_to="none",
    save_strategy="steps",    # Add save strategy
    save_steps=100,           # Save every 100 steps
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_args,
)

# 6. Eğitimi Başlatma
trainer.train()

# Check if the adapter_config.json file exists
adapter_path = os.path.join(output_dir, "adapter_config.json")
print(f"Checking if adapter config exists at: {adapter_path}")
print(f"File exists: {os.path.exists(adapter_path)}")

# List all files in output directory to debug
print("Files in output directory:")
print(os.listdir(output_dir))

# Find the latest checkpoint directory
checkpoint_dirs = [d for d in os.listdir(output_dir) if d.startswith("checkpoint")]
if checkpoint_dirs:
    latest_checkpoint = max(checkpoint_dirs, key=lambda x: int(x.split("-")[-1]))
    adapter_dir = os.path.join(output_dir, latest_checkpoint)
    print(f"Latest checkpoint directory: {adapter_dir}")
else:
    adapter_dir = output_dir
    print(f"No checkpoint directories found, using: {adapter_dir}")

# 7. Modeli Kaydetme ve Yükleme
print(f"Loading adapter from: {adapter_dir}")
peft_model = PeftModel.from_pretrained(model, adapter_dir)
merged_model = peft_model.merge_and_unload()

output_model_dir = "hugin-risale-gemma-3-1b"
# Save merged model locally
merged_model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

# 8. Hugging Face Hub'a Yükleme
merged_model.push_to_hub(output_model_dir)
tokenizer.push_to_hub(output_model_dir)

print("Model successfully trained, merged, and uploaded to Hugging Face Hub!")